In [1]:
import pandas as pd
import unicodedata
import re
import nltk

In [2]:
data_df = pd.read_csv(r"C:\Users\MalindaPieris\Documents\MscResearch\Sinhala-Audio-Classfication-notebooks\notebooks\nlp\dataset-metadata -with-lyrics.csv")
data_df = pd.DataFrame(data_df)

In [3]:
def lineBreakReplace(text):
    return text.replace("\r\n", " ")

In [4]:
data_df['Lyrics'] =data_df.apply(lambda x: lineBreakReplace(x.Lyrics), axis=1)

In [5]:
def removeEscapSequences(text):
    return "".join(ch for ch in text if unicodedata.category(ch)[0]!="C")

In [6]:
data_df['Lyrics'] =data_df.apply(lambda x: removeEscapSequences(x.Lyrics), axis=1)

In [7]:
def remove_digits(text):
    return ''.join([i for i in text if not i.isdigit()])

In [8]:
data_df['Lyrics'] =data_df.apply(lambda x: removeEscapSequences(x.Lyrics), axis=1)

In [9]:
def removeHtmlTags(text):
    p = re.compile(r'<.*?>')
    return p.sub('', text)

In [10]:
data_df['Lyrics'] = data_df.apply(lambda x: removeHtmlTags(x.Lyrics), axis=1)

In [11]:
data_df.head()

,song_id,song_name,Artist,Lyrics,Type
0,C 01,Kaulu Piyan Path Wahanna,Kasun Kalhara,කවුළු පියන්පත් වහන්න සඳළුතලාවේ ඔබ දුරයි නෙතට ම...,Calm
1,C 02,Ruk Aththana Mala Mudune\n,Nanda Malini,රුක් අත්තන මල මුදුනේ බඹරු නටන හැන්දෑවේ… සැදෑ ...,Calm
2,C 03,Samanala Mudune,H R Jothipala and Latha Walpola,සමනළ මුදුනේ සිරිපද සිඹ සිඹ උදා ඉරක් පායයි සැන...,Calm
3,C 04,Niwan Dutu Himi,Victor Rathnayaka,නිවන් දුටු හිමි රැවන් පිළිරැව පමණි මට අද ශේෂ ව...,Calm
4,C 05,Suwanda Danee Danee Danenawa,Rookantha Gunathilake,සුවඳ දැනී දැනී දැනෙනවා... උදා හිරු එළියේ... අළ...,Calm


In [12]:
def removeSpecialCharacters(sentence,keep_apostrophes=True):
    sentence = sentence.strip()
    if keep_apostrophes:
        PATTERN = r'[?|$|&|*|%|!|@|(|)|.|~]'
        filtered_sentence = re.sub(PATTERN, r'', sentence)
        return filtered_sentence
    return sentence

In [13]:
def replace_full_stops(sentence):
    return sentence.replace("."," ")

In [14]:
data_df['Lyrics'] = data_df.apply(lambda x: removeSpecialCharacters(x.Lyrics), axis=1)
data_df['Lyrics'] = data_df.apply(lambda x: replace_full_stops(x.Lyrics), axis=1)

In [15]:
def remove_english_letters(sentence):
    non_english_sentence = re.sub(r'[a-zA-Z]', '', sentence)
    return non_english_sentence

In [16]:
data_df['Lyrics']  = data_df['Lyrics'] .apply(lambda x: remove_english_letters(x))

In [17]:
lines = list(open('stop-words-sinhala.txt',encoding="utf8"))
sinhala_stop_words = [re.sub("\n","",x) for x in lines]

In [18]:
def tokenize_text(text):
    tokens = nltk.word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    return tokens

In [19]:
data_df['token_list']  = data_df['Lyrics'] .apply(lambda x: tokenize_text(x))

In [20]:
def remove_sinhala_stop_words(tokens):
    filtered_tokens = [token for token in tokens if token not in sinhala_stop_words]
    return filtered_tokens

In [21]:
data_df['token_list_after_sw'] = data_df.apply(lambda x: remove_sinhala_stop_words(x.token_list), axis=1)

In [22]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
data_df['pre_processed_stop_words'] = data_df.apply(lambda x: TreebankWordDetokenizer().detokenize(x.token_list_after_sw), axis=1)

In [23]:
from collections import Counter
list_of_words = ' '.join(data_df['pre_processed_stop_words']).split()
unique_words = [word for word, count in Counter(list_of_words).items() if count in [1,2,3,4,5,6,7,8,9]]
def remove_most_common_unique_words(tokens):
    filtered_tokens = [token for token in tokens if token not in unique_words]
    return filtered_tokens

In [24]:
data_df['token_list_after_common_words'] = data_df.apply(lambda x: remove_most_common_unique_words(x.token_list_after_sw), axis=1)

In [25]:
data_df['pre_processed_text'] = data_df.apply(lambda x: TreebankWordDetokenizer().detokenize(x.token_list_after_common_words), axis=1)

In [26]:
list_of_words = ' '.join(data_df['pre_processed_text']).split()
Counter(list_of_words)

Counter({'ඔබ': 237,
         'නෙතට': 10,
         'මා': 256,
         'තලාවේ': 10,
         'මල්': 130,
         'බලා': 31,
         'දිය': 33,
         'සුවඳ': 56,
         'දුර': 11,
         'සිනා': 44,
         'තරු': 31,
         'සිත': 41,
         'ඈත': 18,
         'මල': 18,
         'මුදුනේ': 10,
         'බඹරු': 10,
         'අඳුර': 11,
         'කර': 23,
         'කවුරුද': 14,
         'එන්නේ…': 10,
         'නුවන්': 23,
         'මුතු': 23,
         'පිණි': 11,
         'හිරු': 14,
         'කියා': 30,
         'අප': 44,
         'පුරා': 15,
         'සිඹ': 10,
         'සුව': 24,
         'දෙන': 14,
         'දැහැමි': 10,
         'දැයක්': 10,
         'පිබිදෙයි': 10,
         'සුදු': 32,
         'රට': 11,
         'දුටු': 15,
         'මට': 133,
         'අද': 35,
         'මැනවී': 17,
         'බුදු': 14,
         'කරැණාවෙන්': 21,
         'වී': 34,
         'ඔබේ': 88,
         'අතින්': 12,
         'වූ': 21,
         'මලක්': 14,
         'හට': 17,
         'මගේ': 147,


In [27]:
data_df['word_count'] = data_df['pre_processed_text'].str.split().str.len()
data_df.head(5)

,song_id,song_name,Artist,Lyrics,Type,token_list,token_list_after_sw,pre_processed_stop_words,token_list_after_common_words,pre_processed_text,word_count
0,C 01,Kaulu Piyan Path Wahanna,Kasun Kalhara,කවුළු පියන්පත් වහන්න සඳළුතලාවේ ඔබ දුරයි නෙතට ම...,Calm,"[කවුළු, පියන්පත්, වහන්න, සඳළුතලාවේ, ඔබ, දුරයි,...","[කවුළු, පියන්පත්, වහන්න, සඳළුතලාවේ, ඔබ, දුරයි,...",කවුළු පියන්පත් වහන්න සඳළුතලාවේ ඔබ දුරයි නෙතට ම...,"[ඔබ, නෙතට, මා, තලාවේ, ඔබ, නෙතට, මා, තලාවේ, මල්...",ඔබ නෙතට මා තලාවේ ඔබ නෙතට මා තලාවේ මල් බලා දිය ...,30
1,C 02,Ruk Aththana Mala Mudune\n,Nanda Malini,රුක් අත්තන මල මුදුනේ බඹරු නටන හැන්දෑවේ… සැදෑ ...,Calm,"[රුක්, අත්තන, මල, මුදුනේ, බඹරු, නටන, හැන්දෑවේ…...","[රුක්, අත්තන, මල, මුදුනේ, බඹරු, නටන, හැන්දෑවේ…...",රුක් අත්තන මල මුදුනේ බඹරු නටන හැන්දෑවේ… සැදෑ අ...,"[මල, මුදුනේ, බඹරු, අඳුර, කර, කවුරුද, එන්නේ…, න...",මල මුදුනේ බඹරු අඳුර කර කවුරුද එන්නේ… නුවන් කවු...,46
2,C 03,Samanala Mudune,H R Jothipala and Latha Walpola,සමනළ මුදුනේ සිරිපද සිඹ සිඹ උදා ඉරක් පායයි සැනස...,Calm,"[සමනළ, මුදුනේ, සිරිපද, සිඹ, සිඹ, උදා, ඉරක්, පා...","[සමනළ, මුදුනේ, සිරිපද, සිඹ, සිඹ, උදා, ඉරක්, පා...",සමනළ මුදුනේ සිරිපද සිඹ සිඹ උදා ඉරක් පායයි සැනස...,"[මුදුනේ, සිඹ, සිඹ, සුව, දෙන, දැහැමි, දැයක්, පි...",මුදුනේ සිඹ සිඹ සුව දෙන දැහැමි දැයක් පිබිදෙයි ද...,58
3,C 04,Niwan Dutu Himi,Victor Rathnayaka,නිවන් දුටු හිමි රැවන් පිළිරැව පමණි මට අද ශේෂ ව...,Calm,"[නිවන්, දුටු, හිමි, රැවන්, පිළිරැව, පමණි, මට, ...","[නිවන්, දුටු, හිමි, රැවන්, පිළිරැව, පමණි, මට, ...",නිවන් දුටු හිමි රැවන් පිළිරැව පමණි මට අද ශේෂ ව...,"[දුටු, මට, අද, මැනවී, බුදු, කරැණාවෙන්, කරැණාවෙ...",දුටු මට අද මැනවී බුදු කරැණාවෙන් කරැණාවෙන් කරැණ...,47
4,C 05,Suwanda Danee Danee Danenawa,Rookantha Gunathilake,සුවඳ දැනී දැනී දැනෙනවා උදා හිරු එළියේ අළුත් ඉර...,Calm,"[සුවඳ, දැනී, දැනී, දැනෙනවා, උදා, හිරු, එළියේ, ...","[සුවඳ, දැනී, දැනී, දැනෙනවා, උදා, හිරු, එළියේ, ...",සුවඳ දැනී දැනී දැනෙනවා උදා හිරු එළියේ අළුත් ඉර...,"[සුවඳ, දැනී, දැනී, හිරු, එළියේ, හිත, හැඟුම්, ස...",සුවඳ දැනී දැනී හිරු එළියේ හිත හැඟුම් සුවඳ දැනී...,48


In [28]:
def total_unique_words(words):
    return nltk.FreqDist(words)

print("Total number of unique words",total_unique_words(list_of_words))

Total number of unique words <FreqDist with 318 samples and 6648 outcomes>


In [29]:
def total_words(words):
    return(len(words))
print("Total number of words",total_words(list_of_words))

Total number of words 6648


In [30]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

In [31]:
X= data_df['pre_processed_text']
y=label_encoder.fit_transform(data_df['Type'])

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=42)

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

countvectorizer = CountVectorizer(analyzer= 'word')
tfidfvectorizer = TfidfVectorizer(analyzer='word')
count_wm = countvectorizer.fit(X)
tfidf_wm = tfidfvectorizer.fit(X)
tfidf_tokens = tfidf_wm.transform(X)
count_tokens = count_wm.transform(X)

In [34]:
from sklearn.model_selection import KFold, cross_val_score

In [35]:
X_train,X_test,y_train,y_test = train_test_split(tfidf_tokens,
                                                y,
                                                test_size=0.33,
                                                random_state=6)

In [36]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [37]:
svm_model = SVC()

param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [0.01, 0.1, 1, 'scale', 'auto'], 'kernel': ['linear', 'rbf', 'poly']}


grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100],
                         'gamma': [0.01, 0.1, 1, 'scale', 'auto'],
                         'kernel': ['linear', 'rbf', 'poly']},
             scoring='accuracy')

In [38]:
from sklearn.metrics import accuracy_score
# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Evaluate the model on the test set using the best hyperparameters
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy*100)

Best Hyperparameters: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
Test Accuracy: 44.44444444444444
